In [1]:


# using Pkg
# Pkg.add("BenchmarkTools")
# Pkg.add("Distributions")
# Pkg.add("CxxWrap")
# Pkg.add("JLD")
# Pkg.add("ImageIO")
# Pkg.add("CircularArrays")
# Pkg.add("OffsetArrays")
# Pkg.add("TensorOperations")
# Pkg.add("TensorCast")
# Pkg.add("Tullio")
# Pkg.add("DiffEqOperators")
# Pkg.add("BandedMatrices")
# Pkg.add("PyPlot")
# Pkg.add("Plots")
# Pkg.add("FFTW")
# Pkg.add("SciPy")
# Pkg.add("GR")
# Pkg.add("Statistics")

import GR
using BenchmarkTools
using Distributions
using Random
using Plots
using PyCall
using PyPlot
#using Images
using CircularArrays
using CxxWrap
using FFTW
using SciPy
using JLD
using Statistics
stats = pyimport("scipy.stats")
np = pyimport("numpy")
cv2 = pyimport("cv2")



PyObject <module 'cv2' from '/usr/lib/python3/dist-packages/cv2.cpython-38-x86_64-linux-gnu.so'>

In [2]:
function Laplacian_2D!(P₁,P₂,A₁,A₂,Δx)
    B₁ = CircularArrays.CircularArray(A₁);
    B₂ = CircularArrays.CircularArray(A₂);
    Threads.@threads for 😄 ∈ CartesianIndices(P₁)
        (i,j) = Tuple(😄)
        P₁[i,j] = @fastmath (-B₁[i+2,j] + 16B₁[i+1,j] 
            + 16B₁[i-1,j] -B₁[i-2,j] 
            -B₁[i,j+2] + 16B₁[i,j+1] 
            + 16B₁[i,j-1] -B₁[i,j-2] 
            - 60B₁[i,j]) /(12Δx^2);
        P₂[i,j] = @fastmath (-B₂[i+2,j] + 16B₂[i+1,j] 
            + 16B₂[i-1,j] -B₂[i-2,j] 
            -B₂[i,j+2] + 16B₂[i,j+1] 
            + 16B₂[i,j-1] -B₂[i,j-2] 
            - 60B₂[i,j]) /(12Δx^2);  
    end
    return nothing
end



function Cores_2D!(N,angle,thr)
    s = []
    count = 0
    accept = 0.5 - 0.5*thr/100
    for 😄 ∈ 1:(N-1)
        for 🥪 ∈ 1:(N-1)
            norm₁ = (angle[😄,🥪] + π)/(2π)
            norm₂ = (angle[😄+1,🥪] + π)/(2π)
            norm₃ = (angle[😄+1,🥪+1] + π)/(2π)
            norm₄ = (angle[😄,🥪+1] + π)/(2π)

            θ₁ = min(abs(norm₂ - norm₁), 1 - abs(norm₂ - norm₁))
            θ₂ = min(abs(norm₃ - norm₂), 1 - abs(norm₃ - norm₂))
            θ₃ = min(abs(norm₄ - norm₃), 1 - abs(norm₄ - norm₃))
            θₛ = θ₁ + θ₂ + θ₃

            if θₛ > accept 
                append!(s,[[😄,🥪]])
            end
        end
    end

    if length(s) > 0
        for 🇸🇦 ∈ 1:(length(s)-1)
  
            diffᵣ = s[🇸🇦 + 1][1] - s[🇸🇦][1]
            diffₛ = s[🇸🇦 + 1][2] - s[🇸🇦][2]

            if diffᵣ == 0 && diffₛ == 1
                count += 1
            end
            if diffᵣ == 1 && diffₛ == 0
                count += 1
            end
        end
    end

    num = length(s) - count

    return num
end



function setting!(moo)
    @inbounds Threads.@threads for t in eachindex(moo)
        if moo[t] > 1
            moo[t] = 1
        end
    end
    return nothing
end

function mooing!(moo,A₁,A₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(moo)
        (i,j) = Tuple(😄)
        @fastmath moo[i,j] = sqrt(A₁[i,j]^2 + A₂[i,j]^2)
    end
    return nothing
end


function angler!(angle,A₁,A₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(angle)
        (i,j) = Tuple(😄)
        angle[i,j] = @fastmath atan(A₂[i,j],A₁[i,j]);
    end
    return nothing
end

function meshgrid(xin,yin)
    nx=length(xin)
    ny=length(yin)
    xout=zeros(ny,nx)
    yout=zeros(ny,nx)
    for jx=1:nx
        for ix=1:ny
            xout[ix,jx]=xin[jx]
            yout[ix,jx]=yin[ix]
        end
    end
    return (x=xout, y=yout)
end


function minmax(A)
    lower = minimum(minimum.(A))
    upper = maximum(maximum.(A))
    return lower,upper
end

minmax (generic function with 1 method)

In [3]:
function fupdate_2D!(F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,Ȧ₁,Ȧ₂,η,λ,fₐ)
    #H = 2.18e-18 
    H = 1
    #a = 4.752e-36 * η^2
    a = η^2
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F₁)
        (i,j) = Tuple(😄)
        F₁[i,j] = @fastmath 1/a^2 *M₁[i,j] .- 3H * Ȧ₁[i,j]  .- A₁[i,j]*(A₁[i,j]^2 .+ A₂[i,j]^2 .- fₐ^2);
        F₂[i,j] = @fastmath 1/a^2 *M₂[i,j] .- 3H * Ȧ₂[i,j]  .- A₂[i,j]*(A₁[i,j]^2 .+ A₂[i,j]^2 .- fₐ^2);
    end
    return nothing
end


function Aupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A₁)
        (i,j) = Tuple(😄)
        A₁[i,j] = @fastmath A₁[i,j] .+ Δt .* (Ȧ₁[i,j] .+ 0.5Δt .* F₁[i,j])
        A₂[i,j] = @fastmath A₂[i,j] .+ Δt .* (Ȧ₂[i,j] .+ 0.5Δt .* F₂[i,j])
    end
    return nothing
end


function velupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,η,λ,fₐ)
    #H = 2.18e-18 
    H = 1
    #a = 4.752e-36 * η^2
    η = η + Δt
    a = η^2

    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ₁)
        (i,j) = Tuple(😄)
        Ȧ₁[i,j] = @fastmath Ȧ₁[i,j] .+ 0.5Δt .* (F₁[i,j] .+ 1/a^2 *M₁[i,j] .- 3H * Ȧ₁[i,j]  .- A₁[i,j]*(A₁[i,j]^2 .+ A₂[i,j]^2 .- fₐ^2));
        Ȧ₂[i,j] = @fastmath Ȧ₂[i,j] .+ 0.5Δt .* (F₂[i,j] .+ 1/a^2 *M₂[i,j] .- 3H * Ȧ₂[i,j]  .- A₂[i,j]*(A₁[i,j]^2 .+ A₂[i,j]^2 .- fₐ^2));
    end
    return nothing
end

function update_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,time,fₐ)

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    fupdate_2D!(F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,Ȧ₁,Ȧ₂,time,1,fₐ)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    Aupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)

    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    
    velupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,time + Δt,1,fₐ)

    return nothing
end



update_2D! (generic function with 1 method)

In [4]:
function run_2D!(n,N,t₀,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,repeats,cont,low,cap)
    #700 images on per 5000 updates
    

    M₁ = zeros(N,N);
    M₂ = zeros(N,N);

    F₁ = zeros(N,N);
    F₂ = zeros(N,N);

    moo = zeros(N,N);
    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    angle = zeros(N,N);

    x = cap

    lo = low
    
    tim = t₀
    
    for 🥪 ∈ 1+cont:repeats+cont
        


        while tim < 20*🥪 + t₀
            tim = round(tim,digits = 4);


            if lo % x == 0
                #125 pictures
                mooing!(moo,A₁,A₂);
                #println(moo[1,1])
                #setting!(moo);
                angler!(angle,A₁,A₂);
                
               PyPlot.imsave("Old/_9moo/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "Greys")
                
                
                PyPlot.imsave("Old/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png",angle,vmin=-π,vmax = π,cmap = "twilight")
                
            end
            
            if tim < 90000
                update_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,tim,fₐ)
                tim = tim + Δt
            end
            lo += 1
        end
     


    end



    return nothing
end

run_2D! (generic function with 1 method)

In [5]:
function PQfupdate_2D!(F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,Ȧ₁,Ȧ₂,η,λ,fₐ)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F₁)
        (i,j) = Tuple(😄)
        F₁[i,j] = @fastmath M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₁[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₁[i,j];
        F₂[i,j] = @fastmath M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₂[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₂[i,j];
    end
    return nothing
end


function PQAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A₁)
        (i,j) = Tuple(😄)
        A₁[i,j] = @fastmath A₁[i,j] .+ Δt .* (Ȧ₁[i,j] .+ 0.5Δt .* F₁[i,j])
        A₂[i,j] = @fastmath A₂[i,j] .+ Δt .* (Ȧ₂[i,j] .+ 0.5Δt .* F₂[i,j])
    end
    return nothing
end


function PQvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,η,λ,fₐ)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ₁)
        (i,j) = Tuple(😄)
        Ȧ₁[i,j] = @fastmath Ȧ₁[i,j] .+ 0.5Δt .* (F₁[i,j] .+ M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₁[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₁[i,j]);
        Ȧ₂[i,j] = @fastmath Ȧ₂[i,j] .+ 0.5Δt .* (F₂[i,j] .+ M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₂[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₂[i,j]);
    end
    return nothing
end

function PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,time,fₐ)

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    PQfupdate_2D!(F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,Ȧ₁,Ȧ₂,time,1,fₐ)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    PQAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)

    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    
    PQvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,time + Δt,1,fₐ)

    return nothing
end



PQupdate_2D! (generic function with 1 method)

In [6]:
function EQCDfupdate_2D!(F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,Ȧ₁,Ȧ₂,η,λ,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F₁)
        (i,j) = Tuple(😄)
        F₁[i,j] = @fastmath M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) + ηₓ^n *η^2 * C₂[i,j]^2/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₁[i,j];
        F₂[i,j] = @fastmath M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - ηₓ^n *η^2 * C₁[i,j]*C₂[i,j]/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₂[i,j];
    end
    return nothing
end


function EQCDAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A₁)
        (i,j) = Tuple(😄)
        A₁[i,j] = @fastmath A₁[i,j] .+ Δt .* (Ȧ₁[i,j] .+ 0.5Δt .* F₁[i,j])
        A₂[i,j] = @fastmath A₂[i,j] .+ Δt .* (Ȧ₂[i,j] .+ 0.5Δt .* F₂[i,j])
    end
    return nothing
end


function EQCDvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,η,λ,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ₁)
        (i,j) = Tuple(😄)
        Ȧ₁[i,j] = @fastmath Ȧ₁[i,j] .+ 0.5Δt .* (F₁[i,j] .+ M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) + ηₓ^n *η^2 * C₂[i,j]^2/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₁[i,j])
        Ȧ₂[i,j] = @fastmath Ȧ₂[i,j] .+ 0.5Δt .* (F₂[i,j] .+ M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - ηₓ^n *η^2 * C₁[i,j]*C₂[i,j]/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₂[i,j])
    end
    return nothing
end


function EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,time,fₐ,r,s,n)



    #λ = (fₐ/mass(fₐ))^2;

    λᵪ = [1024 1448 3072 3584 5504 1];
    #ηₓ,η = ηtime(time,fₐ);
    λ = λᵪ[r]

    ηᵪ = [2.8 2.8 3.2 3.4 3.6 0 400]
    ηₓ = ηᵪ[s]

    if time < ηₓ
        ηₓ = time
    end
    

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    EQCDfupdate_2D!(F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,Ȧ₁,Ȧ₂,time,λ,ηₓ,n)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    EQCDAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)

    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    #ηₓ,η = ηtime(time+Δt,fₐ);

    EQCDvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂, time + Δt,λ,ηₓ,n)

    return nothing
end

EQCDupdate_2D! (generic function with 1 method)

In [7]:
function LLaplacian_2D!(P,A,Δx)
    B = CircularArray(A);
    Threads.@threads for 😄 ∈ CartesianIndices(P)
        (i,j) = Tuple(😄)
        P[i,j] = @fastmath (-B[i+2,j] + 16B[i+1,j] 
            + 16B[i-1,j] -B[i-2,j] 
            -B[i,j+2] + 16B[i,j+1] 
            + 16B[i,j-1] -B[i,j-2] 
            - 60B[i,j]) /(12Δx^2);
    end
    return nothing
end


function Lfupdate_2D!(F,M,A,Ȧ,η,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F)
        (i,j) = Tuple(😄)
        F[i,j] = @fastmath M[i,j] - ηₓ^n * η^2 * sin(A[i,j]) - 2/η * Ȧ[i,j];
    end
    return nothing
end


function LAupdate_2D!(A,Δt,Ȧ,F)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A)
        (i,j) = Tuple(😄)
        A[i,j] = @fastmath A[i,j] .+ Δt .* (Ȧ[i,j] .+ 0.5Δt .* F[i,j])
    end
    return nothing
end


function Lvelupdate_2D!(Ȧ,Δt,F,M,A,η,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ)
        (i,j) = Tuple(😄)
        Ȧ[i,j] = @fastmath Ȧ[i,j] .+ 0.5Δt .* (F[i,j] .+ M[i,j] - ηₓ^n * η^2 * sin(A[i,j]) - 2/η * Ȧ[i,j])
    end
    return nothing
end


function Lupdate_2D!(A,Ȧ,M,F,Δx,Δt,time,fₐ,s,n)

    #ηₓ,η = Lηtime(time,fₐ);


    ηᵪ = [2.8 2.8 3.2 3.4 3.6 0 400]

    ηₓ = ηᵪ[s]

    if time < ηₓ
        ηₓ = time
    end

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    Lfupdate_2D!(F,M,A,Ȧ,time,ηₓ,n)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    LAupdate_2D!(A,Δt,Ȧ,F)

    LLaplacian_2D!(M,A,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    #ηₓ,η = Lηtime(time+Δt,fₐ)

    Lvelupdate_2D!(Ȧ,Δt,F,M,A,time + Δt,ηₓ,n)

    return nothing
end


Lupdate_2D! (generic function with 1 method)

In [8]:
function FPQrun_2D!(n,N,t₀,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,repeats,cont,low)
    #700 images on per 5000 updates
    

    M₁ = zeros(N,N);
    M₂ = zeros(N,N);

    F₁ = zeros(N,N);
    F₂ = zeros(N,N);

    moo = zeros(N,N);
    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    angle = zeros(N,N);

    # fₛ = Δx
    # freq_s = fₛ/2 * range(-1,stop = 1,length = N)
    # freq = fₛ/2 * range(0,stop = 1, length = N/2)
    k_freq = fftfreq(N)*N
    kx,ky = meshgrid(k_freq,k_freq)

    knrm = sqrt.( kx.^2 + ky.^2)
    knrm = collect(Iterators.flatten(knrm))

    kbins = range(0.5, N/2+1, step = 1)
    kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])

    x = 500

    tracker = []
    cores = []
    thr = 1
    
    saxion = []
    axion = []
    axionmax = []
    axionmin = []

    axenergy = []
    
    lo = low
    
    tim = t₀
    
    for 🥪 ∈ 1+cont:repeats+cont
        


        while tim < 100*🥪 + t₀
            tim = round(tim,digits = 8);


            if lo % x == 0
                #125 pictures
                mooing!(moo,A₁,A₂);
                #println(moo[1,1])
                #setting!(moo);
                angler!(angle,A₁,A₂);
                
    

                strng = Cores_2D!(N,angle,thr)
                append!(tracker,tim)
                append!(cores,strng)

                # PyPlot.imsave("PQEpoch/1"*string(i)*"/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")

                f_image = FFTW.fft(moo)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                # Plots.savefig(plotd,"PQEpoch/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                append!(saxion,[Abin])

                f_image = FFTW.fft(angle)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                # Plots.savefig(plotc,"PQEpoch/Angle/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")


                append!(axion,[Abins])
                
                
                
                ρ = ((Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2) ).^2

                rho = mean(ρ)
                
                amax = maximum((log.(ρ/rho)))
                amin = minimum((log.(ρ/rho)))

                append!(axionmax,amax)
                append!(axionmin,amin)


                PyPlot.imsave("PQTOWN/new/O/"*string(i)*"/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png",(log.(ρ/rho)),vmin=0,vmax = 4,cmap = "inferno")


                f_image = FFTW.fft( ρ .* fₐ^2)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                append!(axenergy,[Abint])

                
                
                

                
                
                PyPlot.imsave("PQTOWN/new/"*string(i)*"/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png",angle,vmin=-π,vmax = π,cmap = "twilight")
                
            end
            
            if tim < 250
                PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,tim,fₐ)
                tim = tim + Δt
            elseif tim ≥ 250 && tim < 500
                PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt * 250/tim,tim,fₐ)
                tim = tim + Δt * 250/tim
            else
                PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt * 0.5 * 500/tim,tim,fₐ)
                tim = tim + Δt * 0.5 * 500/tim
            end
            lo += 1
        end
     


        save("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(🥪)*".jld", "Real", A₁,"Imaginary",A₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂)

        save("PQTOWN/new/Saving/"*string(i)*"/MinMax"*string(n)*string(🥪)*".jld", "max", axionmax,"min",axionmin)



        save("PQTOWN/new/Saving/"*string(i)*"/Strings"*string(n)*string(🥪)*".jld","time",tracker,"number",cores,"lo",lo)
        save("PQTOWN/new/Saving/"*string(i)*"/FFT"*string(n)*string(🥪)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy)


        println(tim)
    end



    return nothing
end

FPQrun_2D! (generic function with 1 method)

In [9]:
function FErun_2D!(n,N,t₁,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repeats,cont,low,B,aang,poggers,cap,pic)


    

    M₁ = zeros(N,N);
    M₂ = zeros(N,N);

    F₁ = zeros(N,N);
    F₂ = zeros(N,N);

    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    moo = zeros(N,N);
    angll = zeros(N,N);
    angler!(angll,A₁,A₂);

    k_freq = fftfreq(N)*N
    kx,ky = meshgrid(k_freq,k_freq)

    knrm = sqrt.( kx.^2 + ky.^2)
    knrm = collect(Iterators.flatten(knrm))

    kbins = range(0.5, N/2+1, step = 1)
    kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])
    
    x = cap

    tracker = []
    cores = []
    thr = 1
    lo = low
    strng = 1
    mng = 0

    saxion = []
    axion = []
    axenergy = []
    axionmax = []
    axionmin = []
    
    Ȧ = zeros(N,N)

    Ȧ .= (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)

    M = zeros(N,N);

    F = zeros(N,N);

    LLaplacian_2D!(M,angll,Δx)
    
    tim = t₁
    
    ay,az = minmax(angll)
    star = 3

    
    if B != 0
        Ȧ = B
        strng = 0
        mng = 2
        angll = aang
        ay = 0
        az = 0
    end
    
    
    for 🥪 ∈ 1+cont:repeats+cont
        

        while tim < 0.5*🥪 + t₁
        #while strng != 0
            
            if az ≥ 1 && ay ≤ -1


                tim = round(tim,digits = 7);
                if lo % x == 0
                    #18 pictures
                    mooing!(moo,A₁,A₂);
                    #println(moo[1,1])
                    #setting!(moo);
                    angler!(angll,A₁,A₂);
                    
                    

                    strng = Cores_2D!(N,angll,thr)
                    
          
                    
                    append!(tracker,tim)
                    append!(cores,strng)

                    # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")


                    f_image = FFTW.fft(moo)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                    # Plots.savefig(plotd,"EQCD/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                    append!(saxion,[Abin])

                    f_image = FFTW.fft(angll)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                    # Plots.savefig(plotc,"EQCD/angll/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")

                    append!(axion,[Abins])
                    
                    Ȧ = (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)
                    
                    ρ = Ȧ.^2
                    
                    rho = mean(ρ)
                    
                    amax = maximum((log.(ρ/rho)))
                    amin = minimum((log.(ρ/rho)))
                    
                    append!(axionmax,amax)
                    append!(axionmin,amin)
                    
                    PyPlot.imsave("QCDTOWN/rs/O/"*string(r)*"/"*string(s)*"/"*lpad( string(trunc(Int, pic + lo/x )) ,3,"0")*".png",(log.(ρ/rho)), vmin = 0,vmax = 6,cmap = "inferno")


                    f_image = FFTW.fft( ρ .* fₐ^2)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    append!(axenergy,[Abint])


                    
                    
                    

                    PyPlot.imsave("QCDTOWN/rs/"*string(r)*"/"*string(s)*"/"*lpad( string(trunc(Int, pic+ lo/x )) ,3,"0")*".png",angll,vmin=-π,vmax = π,cmap = "twilight")
                    
                    ay,az = minmax(angll)

                end
                if tim < 3
                
                    EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,tim,fₐ,r,s,poggers)
                    tim = tim + Δt
                    
                else
                    EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt*(3/tim)^3.34,tim,fₐ,r,s,poggers)
                    tim = tim + Δt*(3/tim)^3.34
                    
                end
                lo += 1
                
                


                
            else
                
                if mng == 0
                    Ȧ = zeros(N,N)

                    Ȧ .= (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)

                    M = zeros(N,N);

                    F = zeros(N,N);

                    LLaplacian_2D!(M,angll,Δx)
                elseif mng == 2
                    M = zeros(N,N);

                    F = zeros(N,N);

                    LLaplacian_2D!(M,angll,Δx)
                end


                tim = round(tim,digits = 7);
                if lo % x == 0
                    #13 pictures
                    #mooing!(moo,A₁,A₂);
                    #println(moo[1,1])
                    #setting!(moo);
                    #angllr!(angll,A₁,A₂);

                    strng = Cores_2D!(N,angll,thr)

        

                    append!(tracker,tim)
                    append!(cores,strng)

                    # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")



                    f_image = FFTW.fft(angll)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

#                     plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
#                     Plots.savefig(plotc,"EQCD/angll/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")

                    append!(axion,[Abins])
                    
                    
                    ρ = Ȧ.^2
                    rho = mean(ρ)
                    
                    
                    amax = maximum((log.(ρ/rho)))
                    amin = minimum((log.(ρ/rho)))
                    
                    append!(axionmax,amax)
                    append!(axionmin,amin)
                    
                    
                    PyPlot.imsave("QCDTOWN/rs/O/"*string(r)*"/"*string(s)*"/"*lpad( string(trunc(Int,pic + lo/x )) ,3,"0")*".png",(log.(ρ/rho)),vmin = 0,vmax = 6,cmap = "inferno")


                    f_image = FFTW.fft( ρ .* fₐ^2)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    append!(axenergy,[Abint])





                    

                    PyPlot.imsave("QCDTOWN/rs/"*string(r)*"/"*string(s)*"/"*lpad( string(trunc(Int,pic + lo/x )) ,3,"0")*".png",angll,vmin=-π,vmax = π,cmap = "twilight")
                    
                    ay,az = minmax(angll)

                end
                
                if tim < 3
                    Lupdate_2D!(angll,Ȧ,M,F,Δx,Δt,tim,fₐ,s,poggers)
                    tim = tim + Δt
                else
                    Lupdate_2D!(angll,Ȧ,M,F,Δx,Δt*(3/tim)^3.34,tim,fₐ,s,poggers)
                    tim = tim + Δt*(3/tim)^3.34
                end
                
                lo += 1
   
                mng = 1

               
            end
           

        end

        save("QCDTOWN/rs/Saving/"*string(r)*"/"*string(s)*"/Box"*string(n)*string(🥪)*".jld", "Real", A₁,"Imaginary",A₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂,"angle",angll,"AngVel",Ȧ)
        save("QCDTOWN/rs/Saving/"*string(r)*"/"*string(s)*"/MinMax"*string(n)*string(🥪)*".jld", "max", axionmax,"min",axionmin)



        save("QCDTOWN/rs/Saving/"*string(r)*"/"*string(s)*"/Strings"*string(n)*string(🥪)*".jld","time",tracker,"number",cores,"lo",lo)
        save("QCDTOWN/rs/Saving/"*string(r)*"/"*string(s)*"/FFT"*string(n)*string(🥪)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy)


        println(tim)
 

        
    end


    

    return nothing
end

FErun_2D! (generic function with 1 method)

In [10]:
# using Pkg
# ENV["PYTHON"]="/usr/bin/python3.8"
# Pkg.build("PyCall")


# using CircularArrays
# using JLD
# using FFTW
# #Pkg.add("PyCall")
# #Pkg.build("PyCall")
# #Pkg.add("SciPy")
# #Pkg.add("Conda")
# using Conda
# Conda.add("scipy")
# using PyCall
# stats = pyimport("scipy.stats")
print(real(0 + 1im))

0

In [11]:
#PyPlot.imshow(-2.2*ones(512,512) ,vmin = -3.14, vmax = 3.14, cmap = "twilight")
# cont = 8
# n = 11
# cont = 5
# A = load("QCDTOWN/rs/Saving/5/1/Box1110.jld")["AngVel"]

# ρ = A.^2
# rho = mean(ρ)


# PyPlot.imshow((log.(ρ/rho)),vmin = 0,vmax = 4,cmap = "inferno")


# PyPlot.imsave("QCDTOWN/rs/O/"*string(r)*"/"*string(star)*"/1/"*lpad( string(trunc(Int,pic + lo/x )) ,3,"0")*".png",(log.(ρ/rho)),vmin = 0,vmax = 4,cmap = "inferno")


# C₁ = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Box"*string(n)*string(cont)*".jld")["Real"]
#         C₂ = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Box"*string(n)*string(cont)*".jld")["Imaginary"]
#          Ȧ₁ = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Box"*string(n)*string(cont)*".jld")["Realvel"]
#          Ȧ₂ = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Box"*string(n)*string(cont)*".jld")["RealIm"]

In [11]:
@time begin


    
n = 9
N = 2^n;
println(N)

rₐ = 15
fₐ = 10 .^rₐ;
r = 1
s = 1
    
    
    
LPQ = [125 250 500 1000 2000 4000 8000 16000 32000 64000]



for i ∈ 5:5
        
    cont = 0
    low = 0
    Ȧ = 0
    angle = 0
    cap = 25
    Ȧ = 0
    angll = 0
    pic = 0
    poggers = 3

    t₀ = 0.0001;
    print(string(i)*" ")


    Ȧ₁ = zeros(N,N);
    Ȧ₂ = zeros(N,N);


    μ,σ = 0, 0.1;
    C₁ = rand(Normal(μ,σ),N,N);
    C₂ = rand(Normal(μ,σ),N,N);


    Δx = round(LPQ[6]/N,digits = 4)
    Δt = 0.004#0.000001
    
        repeat = 6
        
        
        
    #low = load("PQTOWN/new/Saving/"*string(i)*"/Strings"*string(n)*string(repeat)*".jld")["lo"]
        


       FPQrun_2D!(n, N, t₀, C₁, C₂, Ȧ₁, Ȧ₂, Δx, Δt,fₐ,i,repeat,cont,low)
        

        s = 5
        r = 5


            
            


        

#         #t₁ = t₀ + repeat * 100

#         LQCD = LPQ/(repeat*100)*0.4

#         Δx = round(LQCD[6]/N,digits = 6)
#         Δt = 0.001

        
#         #cont = 90
#                 C₁ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Real"]
#         C₂ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Imaginary"]
#          Ȧ₁ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Realvel"]
#          Ȧ₂ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["RealIm"]
#         t₁ = 0.4 + 0.5*cont
        
#         #low = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Strings"*string(n)*string(cont)*".jld")["lo"]
        
        
# #             C₁ = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Box"*string(n)*string(cont)*".jld")["Real"]
# #         C₂ = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Box"*string(n)*string(cont)*".jld")["Imaginary"]
# #          Ȧ₁ = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Box"*string(n)*string(cont)*".jld")["Realvel"]
# #          Ȧ₂ = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Box"*string(n)*string(cont)*".jld")["RealIm"]

#         repeat = 20


#          FErun_2D!(n,N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repeat,cont,low,Ȧ,angll,poggers,cap,pic)
        
      


end
    
    

    println(' ')
end

512
5 100.0001
200.0001
300.0003868060732
400.0006492615672
500.0007425650298
600.0007713091538
 
5669.527084 seconds (35.12 M allocations: 26.553 GiB, 0.03% gc time)


In [ ]:
@time begin


    
n = 9
N = 2^n;
println(N)

rₐ = 15
fₐ = 10 .^rₐ;
r = 1
s = 1
    
    
    
LPQ = [125 250 500 1000 2000 4000 8000 16000 32000 64000]



for i ∈ 5:5
        
    cont = 0
    low = 0
    Ȧ = 0
    angle = 0
    cap = 25
    Ȧ = 0
    angll = 0
    pic = 0
    poggers = 6.68

    t₀ = 0.0001;
    print(string(i)*" ")


    Ȧ₁ = zeros(N,N);
    Ȧ₂ = zeros(N,N);


    μ,σ = 0, 0.1;
    C₁ = rand(Normal(μ,σ),N,N);
    C₂ = rand(Normal(μ,σ),N,N);


    Δx = 4/N#round(LPQ[1]/N,digits = 4)
    Δt = 0.000001
    
        repeat = 4
        
        
        
    #low = load("PQTOWN/new/Saving/"*string(i)*"/Strings"*string(n)*string(repeat)*".jld")["lo"]
        
#         C₁ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Real"]
#         C₂ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Imaginary"]
#          Ȧ₁ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Realvel"]
#          Ȧ₂ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["RealIm"]

       FPQrun_2D!(n, N, t₀, C₁, C₂, Ȧ₁, Ȧ₂, Δx, Δt,fₐ,i,repeat,cont,low)
        

        s = 6
        r = 6


            
            


        

        #t₁ = t₀ + repeat * 100

        #LQCD = LPQ/(repeat*100)*0.4

        #Δx = round(LQCD[6]/N,digits = 6)
        Δt = 0.001

        
        #cont = 90
        t₁ = 0.4 + 0.5*cont
        
        #low = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Strings"*string(n)*string(cont)*".jld")["lo"]
        
        
            #C₁ = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Box"*string(n)*string(cont)*".jld")["Real"]
        #C₂ = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Box"*string(n)*string(cont)*".jld")["Imaginary"]
         #Ȧ₁ = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Box"*string(n)*string(cont)*".jld")["Realvel"]
         #Ȧ₂ = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Box"*string(n)*string(cont)*".jld")["RealIm"]

        repeat = 10


#         FErun_2D!(n,N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repeat,cont,low,Ȧ,angll,poggers,cap,pic)
        
      


end
    
    

    println(' ')
end

512
5 

In [ ]:
#do many box simulations starting from PQ next to find the axion mass

In [ ]:
minn = load("PQTOWN/Saving/5/MinMax52.jld")["min"]
println(minimum(minn))
println(mean(minn))
Plots.plot(minn)

In [ ]:
maxx = load("PQTOWN/Saving/5/MinMax52.jld")["max"]
println(maximum(maxx))
println(mean(maxx))
Plots.plot(maxx)

In [ ]:
minn = load("QCDTOWN/Saving/5/MinMax51611.jld")["min"]
println(minimum(minn))
println(mean(minn))
Plots.plot(minn)


In [ ]:
maxx = load("QCDTOWN/Saving/5/MinMax51611.jld")["max"]
println(maximum(maxx))
println(mean(maxx))
Plots.plot(maxx)

In [ ]:

load("PQTOWN/Saving/1/Strings53.jld")["time"]
load("QCDTOWN/Saving/1/Strings51.jld")["time"]

In [ ]:
"""λᵪ = [1024 1448 3072 3584 5504];
#ηₓ,η = ηtime(time,fₐ);
λ = λᵪ[r]

ηᵪ = range(2.8,stop = 3.6,length = 5)
ηₓ = ηᵪ[s]"""

In [ ]:
runlong time
change lambda =1 tp something

oscillons find

n = 6.68

strings see if attaract construcitons

In [26]:

function meshgrid(xin,yin)
    nx=length(xin)
    ny=length(yin)
    xout=zeros(ny,nx)
    yout=zeros(ny,nx)
    for jx=1:nx
        for ix=1:ny
            xout[ix,jx]=xin[jx]
            yout[ix,jx]=yin[ix]
        end
    end
    return (x=xout, y=yout)
end

meshgrid (generic function with 1 method)

In [10]:
n = 9

N = 2^n

k_freq = fftfreq(N)*N
kx,ky = meshgrid(k_freq,k_freq)

knrm = sqrt.( kx.^2 + ky.^2)
knrm = collect(Iterators.flatten(knrm))

kbins = range(0.5, N/2+1, step = 1)
kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])

1.0:1.0:256.0

In [15]:


sax1 = load("QCDTOWN/rs/Saving/1/1/1/FFT990.jld")["saxion"]
ax1 = load("QCDTOWN/rs/Saving/1/1/1/FFT990.jld")["axion"]

    while length(sax1) != length(ax1)
        append!(sax1,[range(0,0,length = length(sax1[1])) ])
    end

tracker = load("QCDTOWN/rs/Saving/1/1/1/Strings990.jld")["time"]




for i ∈ 1:length(sax1)

    plotc = Plots.plot!(kvals,sax1[i],xaxis =:log ,yaxis =:log,legend = true,label = "ηᵪ = 0" ,ylim = (1e2,1e6),title = "2048: λ = 1024 ,Saxion η = "*string(tracker[i]))



    Plots.savefig(plotc,"QCDTOWN/rs/Saving/Saxion/Long/"*lpad( string(i-1) ,3,"0")*".png")

    plotc = Plots.plot()

end


for i ∈ 1:length(ax1)

    plotc = Plots.plot!(kvals,ax1[i],xaxis =:log ,yaxis =:log,legend = true,label = "ηᵪ = 0" ,title = "2048: λ = 1024 ,Axion η = "*string(tracker[i]))


    Plots.savefig(plotc,"QCDTOWN/rs/Saving/Axion/Long/"*lpad( string(i-1) ,3,"0")*".png")

    plotc = Plots.plot()

end


# for i ∈ 1:length(axion[1])
#     plotc = Plots.plot(kvals,[axion[1][i],axion[2][i],axion[3][i],axion[4][i]],xaxis =:log ,yaxis =:log,legend = true,label = ["ηᵪ = 2.8" "ηᵪ = 3.2" "ηᵪ = 3.4" "ηᵪ = 3.6"] ,title = "2048: λ = 1024 ,Axion η = "*string(tracker[i]))
#     Plots.savefig(plotc,"QCDTOWN/rs/Saving/Axion/eta/"*lpad( string(i-1) ,3,"0")*".png")

# end


┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
GKS: Possible loss of precision in routine SET_WINDOW
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks

┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils

┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils

┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils

┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils

...

┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils

┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils

┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils

┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils

┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/es5pb/src/ticks.jl:283
┌ Warning: No strict ticks found
└ @ PlotUtils

In [ ]:


sax1 = load("QCDTOWN/rs/Saving/1/1/FFT1116.jld")["saxion"]
ax1 = load("QCDTOWN/rs/Saving/1/1/FFT1116.jld")["axion"]



sax2 = load("QCDTOWN/rs/Saving/1/3/FFT1116.jld")["saxion"]
ax2 = load("QCDTOWN/rs/Saving/1/3/FFT1116.jld")["axion"]


sax3 = load("QCDTOWN/rs/Saving/1/4/FFT1116.jld")["saxion"]
ax3 = load("QCDTOWN/rs/Saving/1/4/FFT1116.jld")["axion"]

sax4 = load("QCDTOWN/rs/Saving/1/5/FFT1116.jld")["saxion"]
ax4 = load("QCDTOWN/rs/Saving/1/5/FFT1116.jld")["axion"]

tracker = load("QCDTOWN/rs/Saving/1/1/Strings1116.jld")["time"]






# for i ∈ 1:length(sax4)

#     plotc = Plots.plot!(kvals,sax1[i],xaxis =:log ,yaxis =:log,legend = true,label = "ηᵪ = 2.8" , ylims = (10^4,10^14),title = "2048: λ = 1024 ,Saxion η = "*string(tracker[i]))
#     plotc = Plots.plot!(kvals,sax2[i], label = "ηᵪ = 3.2")
#     plotc = Plots.plot!(kvals,sax3[i], label = "ηᵪ = 3.4")
#     plotc = Plots.plot!(kvals,sax4[i], label = "ηᵪ = 3.6")
    


#     Plots.savefig(plotc,"QCDTOWN/rs/Saving/Saxion/eta/"*lpad( string(i-1) ,3,"0")*".png")

#     plotc = Plots.plot()

# end


for i ∈ 1:length(ax4)

    plotc = Plots.plot!(kvals,ax1[i],xaxis =:log ,yaxis =:log,legend = true,label = "ηᵪ = 2.8" , ylims = (10^4,10^13),title = "2048: λ = 1024 ,Axion η = "*string(tracker[i]))
    plotc = Plots.plot!(kvals,ax2[i], label = "ηᵪ = 3.2")
    plotc = Plots.plot!(kvals,ax3[i], label = "ηᵪ = 3.4")
    plotc = Plots.plot!(kvals,ax4[i], label = "ηᵪ = 3.6")
    


    Plots.savefig(plotc,"QCDTOWN/rs/Saving/Axion/eta/"*lpad( string(i-1) ,3,"0")*".png")

    plotc = Plots.plot()

end


# for i ∈ 1:length(axion[1])
#     plotc = Plots.plot(kvals,[axion[1][i],axion[2][i],axion[3][i],axion[4][i]],xaxis =:log ,yaxis =:log,legend = true,label = ["ηᵪ = 2.8" "ηᵪ = 3.2" "ηᵪ = 3.4" "ηᵪ = 3.6"] ,title = "2048: λ = 1024 ,Axion η = "*string(tracker[i]))
#     Plots.savefig(plotc,"QCDTOWN/rs/Saving/Axion/eta/"*lpad( string(i-1) ,3,"0")*".png")

# end


In [18]:


sax1 = load("QCDTOWN/rs/Saving/1/1/FFT1116.jld")["saxion"]
ax1 = load("QCDTOWN/rs/Saving/1/1/FFT1116.jld")["axion"]


# sax2 = load("QCDTOWN/rs/Saving/2/1/FFT1116.jld")["saxion"]
# ax2 = load("QCDTOWN/rs/Saving/2/1/FFT1116.jld")["axion"]



sax3 = load("QCDTOWN/rs/Saving/3/1/FFT1116.jld")["saxion"]
ax3 = load("QCDTOWN/rs/Saving/3/1/FFT1116.jld")["axion"]

sax4 = load("QCDTOWN/rs/Saving/4/1/FFT1116.jld")["saxion"]
ax4 = load("QCDTOWN/rs/Saving/4/1/FFT1116.jld")["axion"]

sax5 = load("QCDTOWN/rs/Saving/5/1/FFT1116.jld")["saxion"]
ax5 = load("QCDTOWN/rs/Saving/5/1/FFT1116.jld")["axion"]

tracker = load("QCDTOWN/rs/Saving/1/1/Strings1116.jld")["time"]




# for i ∈ 1:80

#     plotc = Plots.plot!(kvals,sax1[i],xaxis =:log ,yaxis =:log,legend = true,label = "λ = 1024" , ylims = (10^2,10^10),title = "2048: ηᵪ = 2.8 ,Saxion η = "*string(tracker[i]))
#     #plotc = Plots.plot!(kvals,sax2[i], label = "λ = 1448")
#     plotc = Plots.plot!(kvals,sax3[i], label = "λ = 3072")
#     plotc = Plots.plot!(kvals,sax4[i], label = "λ = 3584")
#     plotc = Plots.plot!(kvals,sax4[i], label = "λ = 5504")
    


#     Plots.savefig(plotc,"QCDTOWN/rs/Saving/Saxion/lambda/"*lpad( string(i-1) ,3,"0")*".png")

#     plotc = Plots.plot()

# end


for i ∈ 1:length(ax5)

    plotc = Plots.plot!(kvals,ax1[i],xaxis =:log ,yaxis =:log,legend = true,label = "λ = 1024" , ylims = (10^4,10^13),title = "2048: ηᵪ = 2.8 ,Axion η = "*string(tracker[i]))
    #plotc = Plots.plot!(kvals,ax2[i], label = "λ = 1448")
    plotc = Plots.plot!(kvals,ax3[i], label = "λ = 3072")
    plotc = Plots.plot!(kvals,ax4[i], label = "λ = 3584")
    plotc = Plots.plot!(kvals,ax5[i], label = "λ = 5504")
    


    Plots.savefig(plotc,"QCDTOWN/rs/Saving/Axion/lambda/"*lpad( string(i-1) ,3,"0")*".png")

    plotc = Plots.plot()

end


LoadError: BoundsError: attempt to access 256-element StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}} at index [1:1024]

In [ ]:
n = 6.68 in folder 5
n = 1 in folder 4
n = 2 in folder 1
n = 3 in folder 2
n = 4 in folder 3
n = 5 in folder 7
n = 6 in folder 8

n = 6.68 in folder 6 but longer continuation but less frames

In [36]:
n = 9

N = 2^n

k_freq = fftfreq(N)*N
kx,ky = meshgrid(k_freq,k_freq)

knrm = sqrt.( kx.^2 + ky.^2)
knrm = collect(Iterators.flatten(knrm))

kbins = range(0.5, N/2+1, step = 1)
kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])

1.0:1.0:256.0

In [37]:
 sax1 = load("QCDTOWN/rs/Saving/1/1/5/FFT112.jld")["saxion"]
 ax1 = load("QCDTOWN/rs/Saving/1/1/5/FFT112.jld")["axion"]
tracker1 = load("QCDTOWN/rs/Saving/1/1/5/Strings112.jld")["time"]
axenergy1 = load("QCDTOWN/rs/Saving/1/1/5/FFT112.jld")["axenergy"]

 sax2 = load("QCDTOWN/rs/Saving/1/1/5/FFT113.jld")["saxion"]
# ax2 = load("QCDTOWN/rs/Saving/1/1/5/FFT113.jld")["axion"]
tracker2 = load("QCDTOWN/rs/Saving/1/1/5/Strings113.jld")["time"]
axenergy2 = load("QCDTOWN/rs/Saving/1/1/5/FFT113.jld")["axenergy"]

 sax3 = load("QCDTOWN/rs/Saving/1/1/5/FFT115.jld")["saxion"]
# ax3 = load("QCDTOWN/rs/Saving/1/1/5/FFT115.jld")["axion"]
tracker3 = load("QCDTOWN/rs/Saving/1/1/5/Strings115.jld")["time"]
axenergy3 = load("QCDTOWN/rs/Saving/1/1/5/FFT115.jld")["axenergy"]

sax = []
ax = []
tracker = []
axenergy = []


 append!(sax,sax1,sax2,sax3)
# append!(ax,ax1,ax2,ax3)
append!(tracker,tracker1,tracker2,tracker3)
append!(axenergy,axenergy1,axenergy2,axenergy3)

 sacs1 = load("QCDTOWN/rs/Saving/1/1/4/FFT114.jld")["saxion"]
# acs1 = load("QCDTOWN/rs/Saving/1/1/4/FFT114.jld")["axion"]
tracks1 = load("QCDTOWN/rs/Saving/1/1/4/Strings114.jld")["time"]
axs1 = load("QCDTOWN/rs/Saving/1/1/4/FFT114.jld")["axenergy"]

 sacs2 = load("QCDTOWN/rs/Saving/1/1/4/FFT115.jld")["saxion"]
# acs2 = load("QCDTOWN/rs/Saving/1/1/4/FFT115.jld")["axion"]
tracks2 = load("QCDTOWN/rs/Saving/1/1/4/Strings115.jld")["time"]
axs2 = load("QCDTOWN/rs/Saving/1/1/4/FFT115.jld")["axenergy"]

sacs = []
acs = []
tracks = []
axs = []

 append!(sacs,sacs1,sacs2)
# append!(acs,acs1,acs2)
append!(tracks,tracks1,tracks2)
append!(axs,axs1,axs2)


# println(length(sax))
# println(length(sacs))
# println(length(tracker))


 sanny2 = load("QCDTOWN/rs/Saving/1/1/1/FFT115.jld")["saxion"]
# anny2 = load("QCDTOWN/rs/Saving/1/1/1/FFT115.jld")["axion"]
tranny2 = load("QCDTOWN/rs/Saving/1/1/1/Strings115.jld")["time"]
axnny2 = load("QCDTOWN/rs/Saving/1/1/1/FFT115.jld")["axenergy"]

 sanny3 = load("QCDTOWN/rs/Saving/1/1/2/FFT115.jld")["saxion"]
# anny3 = load("QCDTOWN/rs/Saving/1/1/2/FFT115.jld")["axion"]
tranny3 = load("QCDTOWN/rs/Saving/1/1/2/Strings115.jld")["time"]
axnny3 = load("QCDTOWN/rs/Saving/1/1/2/FFT115.jld")["axenergy"]

 sanny4 = load("QCDTOWN/rs/Saving/1/1/3/FFT115.jld")["saxion"]
# anny4 = load("QCDTOWN/rs/Saving/1/1/3/FFT115.jld")["axion"]
tranny4 = load("QCDTOWN/rs/Saving/1/1/3/Strings115.jld")["time"]
axnny4 = load("QCDTOWN/rs/Saving/1/1/3/FFT115.jld")["axenergy"]

 sanny5 = load("QCDTOWN/rs/Saving/1/1/7/FFT115.jld")["saxion"]
# anny5 = load("QCDTOWN/rs/Saving/1/1/7/FFT115.jld")["axion"]
tranny5 = load("QCDTOWN/rs/Saving/1/1/7/Strings115.jld")["time"]
axnny5 = load("QCDTOWN/rs/Saving/1/1/7/FFT115.jld")["axenergy"]

 sanny6 = load("QCDTOWN/rs/Saving/1/1/8/FFT115.jld")["saxion"]
# anny6 = load("QCDTOWN/rs/Saving/1/1/8/FFT115.jld")["axion"]
tranny6 = load("QCDTOWN/rs/Saving/1/1/8/Strings115.jld")["time"]
axnny6 = load("QCDTOWN/rs/Saving/1/1/8/FFT115.jld")["axenergy"]



for i ∈ 1:length(sax)

    plotc = Plots.plot!(kvals,sax[i],xaxis =:log ,yaxis =:log,legend = true,label = "n = 6.68" , ylims = (10^4,10^12),title = "2048: Changing n ,Saxion η = "*string(tracker[i]))

    plotc = Plots.plot!(kvals,sacs[i], label = "n = 1")
    plotc = Plots.plot!(kvals,sanny2[i], label = "n = 2")
    plotc = Plots.plot!(kvals,sanny3[i], label = "n = 3")
    plotc = Plots.plot!(kvals,sanny4[i], label = "n = 4")
    plotc = Plots.plot!(kvals,sanny5[i], label = "n = 5")
    plotc = Plots.plot!(kvals,sanny6[i], label = "n = 6")


    Plots.savefig(plotc,"QCDTOWN/rs/Saving/Saxion/n/"*lpad( string(i-1) ,3,"0")*".png")

    plotc = Plots.plot()

end



# for i ∈ 1:length(ax)

#     plotc = Plots.plot!(kvals,ax[i],xaxis =:log ,yaxis =:log,legend = true,label = "n = 6.68" , ylims = (10^5,10^14),title = "2048: Changing n ,Axion η = "*string(tracker[i]))

#     plotc = Plots.plot!(kvals,acs[i], label = "n = 1")
#     plotc = Plots.plot!(kvals,anny2[i], label = "n = 2")
#     plotc = Plots.plot!(kvals,anny3[i], label = "n = 3")
#     plotc = Plots.plot!(kvals,anny4[i], label = "n = 4")
#     plotc = Plots.plot!(kvals,anny5[i], label = "n = 5")
#     plotc = Plots.plot!(kvals,anny6[i], label = "n = 6")

    

#     Plots.savefig(plotc,"QCDTOWN/rs/Saving/Axion/n/"*lpad( string(i-1) ,3,"0")*".png")

#     plotc = Plots.plot()

# end


# for i ∈ 1:length(axenergy)

#     plotc = Plots.plot!(kvals,axenergy[i],xaxis =:log ,yaxis =:log,legend = true,label = "n = 6.68" ,ylims = (1,1e30), title = "2048: Axion Energy Density η = "*string(tracker[i]))

#     plotc = Plots.plot!(kvals,axs[i], label = "n = 1")
#     plotc = Plots.plot!(kvals,axnny2[i], label = "n = 2")
#     plotc = Plots.plot!(kvals,axnny3[i], label = "n = 3")
#     plotc = Plots.plot!(kvals,axnny4[i], label = "n = 4")
#     plotc = Plots.plot!(kvals,axnny5[i], label = "n = 5")
#     plotc = Plots.plot!(kvals,axnny6[i], label = "n = 6")


#     Plots.savefig(plotc,"QCDTOWN/rs/Saving/Axenergy/n/"*lpad( string(i-1) ,3,"0")*".png")

#     plotc = Plots.plot()

# end



LoadError: BoundsError: attempt to access 256-element StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}} at index [1:1024]

In [ ]:
saxion = load("3D/QCDTOWN/Saving/7/FFT53.jld")["saxion"]
axion = load("3D/QCDTOWN/Saving/7/FFT53.jld")["axion"]
axenergy = load("3D/QCDTOWN/Saving/7/FFT53.jld")["axenergy"]
tracker = load("3D/QCDTOWN/Saving/7/Strings53.jld")["time"]
cores = load("3D/QCDTOWN/Saving/7/Strings53.jld")["number"]

lowₛ = minimum(minimum(saxion))
highₛ = maximum(maximum(saxion))
lowₐ = minimum(minimum(axion))
highₐ = maximum(maximum(axion))
lowₑ = minimum(minimum(axenergy))
highₑ = maximum(maximum(axenergy))

for i ∈ 1:length(saxion)

    plotc = Plots.plot(kvals,saxion[i],xaxis =:log ,yaxis =:log,legend = false, ylims = (lowₛ,highₛ),title = "λ = 5504 ,ηᵪ = 2.8 ,Saxion η = "*string(tracker[i]))
    Plots.savefig(plotc,"3D/QCDTOWN/Saving/7/Saxion/"*lpad( string(i-1) ,3,"0")*".png")
    
end

for i ∈ 1:length(axion)

    plotd = Plots.plot(kvals,axion[i],xaxis =:log ,yaxis =:log,legend = false, ylims = (lowₐ,highₐ),title = "λ = 5504 ,ηᵪ = 2.8 ,Axion (a/fₐ) η = "*string(tracker[i]))
    Plots.savefig(plotd,"3D/QCDTOWN/Saving/7/Axion/"*lpad( string(i-1) ,3,"0")*".png")

end

for i ∈ 1:length(axenergy)

    plote = Plots.plot(kvals,axenergy[i]/1e30,xaxis =:log ,yaxis = :log,title = "Axion Energy Density (ρₐ = ȧ^2 / fₐ^2) η = "*string(tracker[i]),legend = false,ylims = (lowₑ/1e30,highₑ/1e30))
    Plots.savefig(plote,"3D/QCDTOWN/Saving/7/Axenergy/"*lpad( string(i-1) ,3,"0")*".png")

end



plotf = Plots.plot(tracker,cores,title = "λ = 1024 ,ηᵪ = 2.8 ,String Count",legend = false)
Plots.savefig(plotf,"3D/QCDTOWN/Saving/7/Strings.png")


In [8]:
@time begin


    
n = 11
N = 2^n;
println(N)

rₐ = 15
fₐ = 10 .^rₐ;
r = 1
s = 1
    
    
LPQ = [125 250 500 1000 2000 4000 8000 16000 32000 64000]

pog = [2 3 4 5 6]
folder = [1 2 3 7 8]

for i ∈ 1:1
        
    angll = 0
    Ȧ = 0
    r = 1
    s = 1
        
    #poggers = pog[i]
    #fold = folder[i]
    fold = 5
    poggers = 6.68
    cont = 5
    #low = load("QCDTOWN/rs/Saving/1/1/5/Strings115.jld")["lo"]
    low = 0
    cap = 30
    pic = 0

#     t₀ = 0.0001;
#     print(string(i)*" ")


#     Ȧ₁ = zeros(N,N);
#     Ȧ₂ = zeros(N,N);


#     μ,σ = 0, 0.1;
#     C₁ = rand(Normal(μ,σ),N,N);
#     C₂ = rand(Normal(μ,σ),N,N);


    Δx = round(LPQ[6]/N,digits = 4)
    Δt = 0.004
        
    repeat = 2
        

     #FPQrun_2D!(n, N, t₀, C₁, C₂, Ȧ₁, Ȧ₂, Δx, Δt,fₐ,i,repeat,cont,low)
        

#     C₁ = load("PQTOWN/Saving/6/Box"*string(n)*string(repeat)*".jld")["Real"]
#     C₂ = load("PQTOWN/Saving/6/Box"*string(n)*string(repeat)*".jld")["Imaginary"]
#      Ȧ₁ = load("PQTOWN/Saving/6/Box"*string(n)*string(repeat)*".jld")["Realvel"]
#      Ȧ₂ = load("PQTOWN/Saving/6/Box"*string(n)*string(repeat)*".jld")["RealIm"]

    C₁ = load("QCDTOWN/rs/Saving/1/1/5/Box115.jld")["Real"]
    C₂ = load("QCDTOWN/rs/Saving/1/1/5/Box115.jld")["Imaginary"]
     Ȧ₁ = load("QCDTOWN/rs/Saving/1/1/5/Box115.jld")["Realvel"]
     Ȧ₂ = load("QCDTOWN/rs/Saving/1/1/5/Box115.jld")["RealIm"]
    #angle = load("QCDTOWN/rs/Saving/1/1/5/Box1116.jld")["Angle"]
    #Ȧ = load("QCDTOWN/rs/Saving/1/1/5/Box1116.jld")["AngVel"]
    
    #t₁ = load("QCDTOWN/Saving/"*string(i)*"/Strings"*string(n)*string(cont)*string(r)*string(s)*".jld")["time"]
    t₁ = 0.4 + 0.5 * cont   
        
    LQCD = LPQ/(repeat*100)*0.4
        
    Δx = round(LQCD[6]/N,digits = 6)
    Δt = 0.001
        
    repeat = 10
        
    FErun_2D!(n,N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,fold,r,s,repeat,cont,low,Ȧ,angll,poggers,cap,pic)


end
    

    println(' ')
end

2048
3.400303962047354
3.9000883388514933
4.4001277932465905
4.900074152061758
5.400080212575018
5.900103557467666
6.400079005672146
6.900027020772319
7.400039917688841
7.900017101667377
 
46031.178785 seconds (28.75 M allocations: 1.117 TiB, 0.15% gc time, 0.00% compilation time)


In [ ]:
C₁ = rand(Normal(0,1),3,3);
print(minimum(minimum.(C₁)))

In [6]:
function PQFFT_2D!(n,N,t₀,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,repeats,cont,low)
    #700 images on per 5000 updates
    

    M₁ = zeros(N,N);
    M₂ = zeros(N,N);

    F₁ = zeros(N,N);
    F₂ = zeros(N,N);

    moo = zeros(N,N);
    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    angle = zeros(N,N);

    # fₛ = Δx
    # freq_s = fₛ/2 * range(-1,stop = 1,length = N)
    # freq = fₛ/2 * range(0,stop = 1, length = N/2)
    k_freq = fftfreq(N)*N
    kx,ky = meshgrid(k_freq,k_freq)

    knrm = sqrt.( kx.^2 + ky.^2)
    knrm = collect(Iterators.flatten(knrm))

    kbins = range(0.5, N/2+1, step = 1)
    kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])

    x = 500

    tracker = []
    cores = []
    thr = 1
    
    saxion = []
    axion = []
    axionmax = []
    axionmin = []

    axenergy = []
    
    lo = low
    
    tim = t₀
    
    for 🥪 ∈ 1+cont:repeats+cont
        


        while tim < 200*🥪 + t₀
            tim = round(tim,digits = 4);


            if lo % x == 0
                #125 pictures
                mooing!(moo,A₁,A₂);
                #println(moo[1,1])
                #setting!(moo);
                angler!(angle,A₁,A₂);
                
    

                strng = Cores_2D!(N,angle,thr)
                append!(tracker,tim)
                append!(cores,strng)

                # PyPlot.imsave("PQEpoch/1"*string(i)*"/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")

                f_image = FFTW.fft(moo)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                # Plots.savefig(plotd,"PQEpoch/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                append!(saxion,[Abin])

                f_image = FFTW.fft(angle)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                # Plots.savefig(plotc,"PQEpoch/Angle/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")


                append!(axion,[Abins])
                
                
                
                ρ = ((Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2) ).^2

                rho = mean(ρ)
                
                amax = maximum((log.(ρ/rho)))
                amin = minimum((log.(ρ/rho)))

                append!(axionmax,amax)
                append!(axionmin,amin)


#                 PyPlot.imsave("PQTOWN/new/O/"*string(i)*"/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png",(log.(ρ/rho)),vmin=-20,vmax = 7,cmap = "inferno")


                f_image = FFTW.fft( ρ .* fₐ^2)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                append!(axenergy,[Abint])

                
                
                

                
                
#                 PyPlot.imsave("PQTOWN/new/"*string(i)*"/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png",angle,vmin=-π,vmax = π,cmap = "twilight")
                
            end
            
            if tim < 250
                PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,tim,fₐ)
                tim = tim + Δt
            elseif tim ≥ 250 && tim < 500
                PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt * 250/tim,tim,fₐ)
                tim = tim + Δt * 250/tim
            else
                PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt * 0.5 * 500/tim,tim,fₐ)
                tim = tim + Δt * 0.5 * 500/tim
            end
            lo += 1
        end
     


        save("PQTOWN/FFT/Box"*string(i)*string(n)*string(🥪)*".jld", "Real", A₁,"Imaginary",A₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂)

        save("PQTOWN/FFT/MinMax"*string(i)*string(n)*string(🥪)*".jld", "max", axionmax,"min",axionmin)



        save("PQTOWN/FFT/Strings"*string(i)*string(n)*string(🥪)*".jld","time",tracker,"number",cores,"lo",lo)
        save("PQTOWN/FFT/FFT"*string(i)*string(n)*string(🥪)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy)



    end



    return nothing
end

PQFFT_2D! (generic function with 1 method)

In [7]:
function QCDFFT_2D!(n,N,t₁,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repeats,cont,low,B,aang,poggers,cap,pic)


    

    M₁ = zeros(N,N);
    M₂ = zeros(N,N);

    F₁ = zeros(N,N);
    F₂ = zeros(N,N);

    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    moo = zeros(N,N);
    angll = zeros(N,N);
    angler!(angll,A₁,A₂);

    k_freq = fftfreq(N)*N
    kx,ky = meshgrid(k_freq,k_freq)

    knrm = sqrt.( kx.^2 + ky.^2)
    knrm = collect(Iterators.flatten(knrm))

    kbins = range(0.5, N/2+1, step = 1)
    kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])
    
    x = cap

    tracker = []
    cores = []
    thr = 1
    lo = low
    strng = 1
    mng = 0

    saxion = []
    axion = []
    axenergy = []
    axionmax = []
    axionmin = []
    
    Ȧ = zeros(N,N)

    Ȧ .= (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)

    M = zeros(N,N);

    F = zeros(N,N);

    LLaplacian_2D!(M,angll,Δx)
    
    tim = t₁
    
    ay,az = minmax(angll)
    
    
    if B != 0
        Ȧ = B
        strng = 0
        mng = 2
        angll = aang
        ay = 0
        az = 0
    end
    
    
    for 🥪 ∈ 1+cont:repeats+cont
        

        while tim < 7*🥪 + 0.4
        #while strng != 0
            
            if az ≥ 1 && ay ≤ -1


                tim = round(tim,digits = 7);
                if lo % x == 0
                    #18 pictures
                    mooing!(moo,A₁,A₂);
                    #println(moo[1,1])
                    #setting!(moo);
                    angler!(angll,A₁,A₂);
                    
                    

                    strng = Cores_2D!(N,angll,thr)
                    
          
                    
                    append!(tracker,tim)
                    append!(cores,strng)

                    # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")


                    f_image = FFTW.fft(moo)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                    # Plots.savefig(plotd,"EQCD/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                    append!(saxion,[Abin])

                    f_image = FFTW.fft(angll)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                    # Plots.savefig(plotc,"EQCD/angll/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")

                    append!(axion,[Abins])
                    
                    
                    ρ = ((Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2) ).^2
                    
                    rho = mean(ρ)
                    
                    amax = maximum((log.(ρ/rho)))
                    amin = minimum((log.(ρ/rho)))
                    
                    append!(axionmax,amax)
                    append!(axionmin,amin)
                    
#                     PyPlot.imsave("QCDTOWN/rs/O/"*string(r)*"/"*string(s)*"/"*string(i)*"/"*lpad( string(trunc(Int, pic + lo/x )) ,3,"0")*".png",(log.(ρ/rho)), vmin = 0,vmax = 2,cmap = "inferno")


                    f_image = FFTW.fft( ρ .* fₐ^2)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    append!(axenergy,[Abint])


                    
                    
                    

#                     PyPlot.imsave("QCDTOWN/rs/"*string(r)*"/"*string(s)*"/"*string(i)*"/"*lpad( string(trunc(Int, pic+ lo/x )) ,3,"0")*".png",angll,vmin=-π,vmax = π,cmap = "twilight")
                    
                    ay,az = minmax(angll)

                end
                if tim < 3
                
                    EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,tim,fₐ,r,s,poggers)
                    tim = tim + Δt
                    
                else
                    EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt*(3/tim)^3.34,tim,fₐ,r,s,poggers)
                    tim = tim + Δt*(3/tim)^3.34
                    
                end
                lo += 1
                
                


                
            else
                
                if mng == 0
                    Ȧ = zeros(N,N)

                    Ȧ .= (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)

                    M = zeros(N,N);

                    F = zeros(N,N);

                    LLaplacian_2D!(M,angll,Δx)
                elseif mng == 2
                    M = zeros(N,N);

                    F = zeros(N,N);

                    LLaplacian_2D!(M,angll,Δx)
                end


                tim = round(tim,digits = 7);
                if lo % x == 0
                    #13 pictures
                    #mooing!(moo,A₁,A₂);
                    #println(moo[1,1])
                    #setting!(moo);
                    #angllr!(angll,A₁,A₂);

                    strng = Cores_2D!(N,angll,thr)

        

                    append!(tracker,tim)
                    append!(cores,strng)

                    # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")



                    f_image = FFTW.fft(angll)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

#                     plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
#                     Plots.savefig(plotc,"EQCD/angll/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")

                    append!(axion,[Abins])
                    
                    
                    ρ = Ȧ.^2
                    rho = mean(ρ)
                    
                    
                    amax = maximum((log.(ρ/rho)))
                    amin = minimum((log.(ρ/rho)))
                    
                    append!(axionmax,amax)
                    append!(axionmin,amin)
                    
                    
#                     PyPlot.imsave("QCDTOWN/rs/O/"*string(r)*"/"*string(s)*"/"*string(i)*"/"*lpad( string(trunc(Int,pic + lo/x )) ,3,"0")*".png",(log.(ρ/rho)),vmin = 0,vmax = 2,cmap = "inferno")


                    f_image = FFTW.fft( ρ .* fₐ^2)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    append!(axenergy,[Abint])





                    

#                     PyPlot.imsave("QCDTOWN/rs/"*string(r)*"/"*string(s)*"/"*string(i)*"/"*lpad( string(trunc(Int,pic + lo/x )) ,3,"0")*".png",angll,vmin=-π,vmax = π,cmap = "twilight")
                    
                    ay,az = minmax(angll)

                end
                
                if tim < 3
                    Lupdate_2D!(angll,Ȧ,M,F,Δx,Δt,tim,fₐ,s,poggers)
                    tim = tim + Δt
                elseif tim < 8.9
                    Lupdate_2D!(angll,Ȧ,M,F,Δx,Δt*(3/tim)^3.34,tim,fₐ,s,poggers)
                    tim = tim + Δt*(3/tim)^3.34
                else
                    Lupdate_2D!(angll,Ȧ,M,F,Δx,Δt*(3/8.9)^3.34,tim,fₐ,s,poggers)
                    tim = tim + Δt*(3/8.9)^3.34
                end
                
                lo += 1
   
                mng = 1

               
            end
           

        end

        save("QCDTOWN/FFT/Box"*string(i)*string(n)*string(🥪)*".jld", "Real", A₁,"Imaginary",A₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂,"angle",angll,"AngVel",Ȧ)
        #save("QCDTOWN/FFT/MinMax"*string(i)*string(n)*string(🥪)*".jld", "max", axionmax,"min",axionmin)



        save("QCDTOWN/FFT/Strings"*string(i)*string(n)*string(🥪)*".jld","time",tracker,"number",cores,"lo",lo)
        save("QCDTOWN/FFT/FFT"*string(i)*string(n)*string(🥪)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy)


        #println(tim)
 

        
    end


    

    return nothing
end

QCDFFT_2D! (generic function with 1 method)

In [8]:
@time begin


    
n = 7
N = 2^n;
println(N)

rₐ = 15
fₐ = 10 .^rₐ;
r = 1
s = 1
    
    
LPQ = [125 250 500 1000 2000 4000 8000 16000 32000 64000]

pog = [2 3 4 5 6]
folder = [1 2 3 7 8]

for i ∈ 201:300
        
    angll = 0
    Ȧ = 0
    r = 1
    s = 1
        
    #poggers = pog[i]
    #fold = folder[i]
    fold = 5
    poggers = 1
    cont = 0
    #low = load("QCDTOWN/rs/Saving/1/1/5/Strings115.jld")["lo"]
    low = 0
    cap = 30
    pic = 0

    t₀ = 0.0001;
    print(string(i)*" ")


    Ȧ₁ = zeros(N,N);
    Ȧ₂ = zeros(N,N);


    μ,σ = 0, 0.1;
    C₁ = rand(Normal(μ,σ),N,N);
    C₂ = rand(Normal(μ,σ),N,N);


    Δx = round(LPQ[6]/N,digits = 4)
    Δt = 0.004
        
    repeat = 1
        

      PQFFT_2D!(n, N, t₀, C₁, C₂, Ȧ₁, Ȧ₂, Δx, Δt,fₐ,i,repeat,cont,low)
        

    C₁ = load("PQTOWN/FFT/Box"*string(i)*string(n)*string(repeat)*".jld")["Real"]
    C₂ = load("PQTOWN/FFT/Box"*string(i)*string(n)*string(repeat)*".jld")["Imaginary"]
     Ȧ₁ = load("PQTOWN/FFT/Box"*string(i)*string(n)*string(repeat)*".jld")["Realvel"]
     Ȧ₂ = load("PQTOWN/FFT/Box"*string(i)*string(n)*string(repeat)*".jld")["RealIm"]

#     C₁ = load("QCDTOWN/rs/Saving/1/1/5/Box115.jld")["Real"]
#     C₂ = load("QCDTOWN/rs/Saving/1/1/5/Box115.jld")["Imaginary"]
#      Ȧ₁ = load("QCDTOWN/rs/Saving/1/1/5/Box115.jld")["Realvel"]
#      Ȧ₂ = load("QCDTOWN/rs/Saving/1/1/5/Box115.jld")["RealIm"]
    #angle = load("QCDTOWN/rs/Saving/1/1/5/Box1116.jld")["Angle"]
    #Ȧ = load("QCDTOWN/rs/Saving/1/1/5/Box1116.jld")["AngVel"]
    
    #t₁ = load("QCDTOWN/Saving/"*string(i)*"/Strings"*string(n)*string(cont)*string(r)*string(s)*".jld")["time"]
    t₁ = 0.4 + 0.5 * cont   
        
    LQCD = LPQ/(repeat*100)*0.4
        
    Δx = round(LQCD[6]/N,digits = 6)
    Δt = 0.001
        
    repeat = 1
        
    QCDFFT_2D!(n,N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repeat,cont,low,Ȧ,angll,poggers,cap,pic)


end
    

    println(' ')
end

128
201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300  
16809.417030 seconds (441.53 M allocations: 423.497 GiB, 0.13% gc time, 0.00% compilation time)


In [ ]:
#look at 0 to 4 instead of 0 to 2 for axion energy density

#save("QCDTOWN/FFT/Box"*string(i)*string(n)*string(🥪)*".jld", "Real", A₁,"Imaginary",A₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂,"angle",angll,"AngVel",Ȧ)
# 1- 100, r  = 5
# 101 - 200, r =1

In [19]:
function meshgrid(xin,yin)
    nx=length(xin)
    ny=length(yin)
    xout=zeros(ny,nx)
    yout=zeros(ny,nx)
    for jx=1:nx
        for ix=1:ny
            xout[ix,jx]=xin[jx]
            yout[ix,jx]=yin[ix]
        end
    end
    return (x=xout, y=yout)
end

meshgrid (generic function with 1 method)

In [20]:
n = 7

N = 2^n

k_freq = fftfreq(N)*N
kx,ky = meshgrid(k_freq,k_freq)

knrm = sqrt.( kx.^2 + ky.^2)
knrm = collect(Iterators.flatten(knrm))

kbins = range(0.5, N/2+1, step = 1)
kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])

1.0:1.0:64.0

In [35]:
saxion = []
axion = []
axenergy = []
tracker = load("QCDTOWN/FFT/Strings1"*string(n)*"1.jld")["time"]
#cores = []
🐹 = 100

@time begin

for i ∈ 201:200+ 🐹
#     saxioni = load("QCDTOWN/FFT/FFT"*string(i)*string(n)*"1.jld")["saxion"]
#     axioni = load("QCDTOWN/FFT/FFT"*string(i)*string(n)*"1.jld")["axion"]
    axenergyi = load("QCDTOWN/FFT/FFT"*string(i)*string(n)*"1.jld")["axenergy"]
#     while length(saxioni) != length(axioni)
#         append!(saxioni,[range(0,0,length = length(saxioni[1])) ])
#     end
    #append!(cores,[load("QCDTOWN/FFT/Strings"*string(i)*string(n)*"1.jld")["number"]])
#     append!(saxion,[saxioni])
#     append!(axion,[axioni])
    append!(axenergy,[axenergyi])

    
end

# sax = saxion[1]/🐹
# ax = axion[1]/🐹
axe = axenergy[1]/🐹
#cor = cores[1]/🐹
for i ∈ 2:🐹
#     sax = sax .+ saxion[i]/🐹
#     ax = ax .+ axion[i]/🐹
    axe = axe .+ axenergy[i]/🐹
    #cor = cor .+ cores[i]/🐹
end



# for i ∈ 1:length(tracker)

#     plotc = Plots.plot(kvals,sax[i],xaxis =:log ,yaxis =:log,legend = false, ylims = (1e0,1e4),title = "λ = 1024 ,ηᵪ = 2.8 ,Saxion η = "*string(tracker[i]))
#     Plots.savefig(plotc,"QCDTOWN/FFT/Saxion/n=1/"*lpad( string(i-1) ,3,"0")*".png")
    
# end

# for i ∈ 1:length(tracker)

#     plotd = Plots.plot(kvals,ax[i],xaxis =:log ,yaxis =:log,legend = false, ylims = (1e4,1e8),title = "λ = 1024 ,ηᵪ = 2.8 ,Axion (a/fₐ) η = "*string(tracker[i]))
#     Plots.savefig(plotd,"QCDTOWN/FFT/Axion/n=1/"*lpad( string(i-1) ,3,"0")*".png")

    # end 


for i ∈ 1:length(tracker)

    plote = Plots.plot(kvals,axe[i].^2/1e30,xaxis =:log ,yaxis =:log,legend = false,ylim = (1e60,1e68),title = "λ = 1024 ,ηᵪ = 2.8 ,Axion Energy Density (ρ) η = "*string(tracker[i]))
    Plots.savefig(plote,"QCDTOWN/FFT/Energy/n=1/"*lpad( string(i-1) ,3,"0")*".png")

end


end

116.286521 seconds (40.92 M allocations: 1.406 GiB, 3.28% gc time)
